In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from textatistic import Textatistic

import time

ted = pd.read_csv("../../data/ted.csv")
tweets = pd.read_csv("../../data/russian_tweets.csv")
headlines = pd.read_csv("../../data/fakenews.csv")
movie_overviews = pd.read_csv("../../data/movie_overviews.csv")

## BoW model for movie taglines

### In this exercise, you have been provided with a corpus of more than 7000 movie tag lines. Your job is to generate the bag of words representation bow_matrix for these taglines. For this exercise, we will ignore the text preprocessing step and generate bow_matrix directly.

### We will also investigate the shape of the resultant bow_matrix. The first five taglines in corpus have been printed to the console for you to examine.

### Instructions
-    Import the CountVectorizer class from sklearn.
-    Instantiate a CountVectorizer object. Name it vectorizer.
-    Using fit_transform(), generate bow_matrix for corpus.

In [26]:
corpus = movie_overviews[["tagline"]].dropna()
corpus = corpus["tagline"]

In [27]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Create CountVectorizer object
vectorizer = CountVectorizer()

# Generate matrix of word vectors
bow_matrix = vectorizer.fit_transform(corpus)

# Print the shape of bow_matrix
print(bow_matrix.shape)

(7033, 6614)


## Analyzing dimensionality and preprocessing

### In this exercise, you have been provided with a lem_corpus which contains the pre-processed versions of the movie taglines from the previous exercise. In other words, the taglines have been lowercased and lemmatized, and stopwords have been removed.

### Your job is to generate the bag of words representation bow_lem_matrix for these lemmatized taglines and compare its shape with that of bow_matrix obtained in the previous exercise. The first five lemmatized taglines in lem_corpus have been printed to the console for you to examine.

### Instructions
-    Import the CountVectorizer class from sklearn.
-    Instantiate a CountVectorizer object. Name it vectorizer.
-    Using fit_transform(), generate bow_lem_matrix for lem_corpus.

In [30]:
import spacy
nlp = spacy.load("en_core_web_sm")

def lemmize(text):
    doc = nlp(text)
    tmp = ""
    for token in doc:
        tmp += token.lemma_ + " "
    return tmp

lem_corpus = corpus.apply(lemmize)
lem_corpus

1             roll the dice and unleash the excitement ! 
2       still Yelling . still fight . still Ready for ...
3       friend be the people who let you be yourself ....
4       just when his World be back to Normal ... he b...
5                               a Los Angeles Crime Saga 
                              ...                        
9091                      kingsglaive : Final Fantasy xv 
9093    what happen in Vegas , stay in Vegas . unless ...
9095    Decorated Officer . Devoted Family Man . defen...
9097                     a god incarnate . a city doom . 
9098          the band you know . the story you do not . 
Name: tagline, Length: 7033, dtype: object

In [31]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Create CountVectorizer object
vectorizer = CountVectorizer()

# Generate matrix of word vectors
bow_lem_matrix = vectorizer.fit_transform(lem_corpus)

# Print the shape of bow_lem_matrix
print(bow_lem_matrix.shape)

(7033, 5626)


## Mapping feature indices with feature names

### In the lesson video, we had seen that CountVectorizer doesn't necessarily index the vocabulary in alphabetical order. In this exercise, we will learn to map each feature index to its corresponding feature name from the vocabulary.

### We will use the same three sentences on lions from the video. The sentences are available in a list named corpus and has already been printed to the console.

### Instructions
-    Instantiate a CountVectorizer object. Name it vectorizer.
-    Using fit_transform(), generate bow_matrix for corpus.
-    Using the get_feature_names() method, map the column names to the corresponding word in the vocabulary.

In [32]:
corpus = ['The lion is the king of the jungle',
 'Lions have lifespans of a decade',
 'The lion is an endangered species']

In [36]:
# Create CountVectorizer object
vectorizer = CountVectorizer()

# Generate matrix of word vectors
bow_matrix = vectorizer.fit_transform(corpus)

# Convert bow_matrix into a DataFrame
bow_df = pd.DataFrame(bow_matrix.toarray())

# Map the column names to vocabulary 
#bow_df.columns = vectorizer.get_feature_names() ## For some reason CountVectorizer doesn't have get_feature_names
bow_df.columns = vectorizer.get_feature_names_out()

# Print bow_df
print(bow_df)

   an  decade  endangered  have  is  jungle  king  lifespans  lion  lions  of  \
0   0       0           0     0   1       1     1          0     1      0   1   
1   0       1           0     1   0       0     0          1     0      1   1   
2   1       0           1     0   1       0     0          0     1      0   0   

   species  the  
0        0    3  
1        0    0  
2        1    1  


## BoW vectors for movie reviews

### In this exercise, you have been given two pandas Series, X_train and X_test, which consist of movie reviews. They represent the training and the test review data respectively. Your task is to preprocess the reviews and generate BoW vectors for these two sets using CountVectorizer.

### Once we have generated the BoW vector matrices X_train_bow and X_test_bow, we will be in a very good position to apply a machine learning model to it and conduct sentiment analysis.

### Instructions
-    Import CountVectorizer from the sklearn library.
-    Instantiate a CountVectorizer object named vectorizer. Ensure that all words are converted to lowercase and english stopwords are removed.
-    Using X_train, fit vectorizer and then use it to transform X_train to generate the set of BoW vectors X_train_bow.
-    Transform X_test using vectorizer to generate the set of BoW vectors X_test_bow.

In [50]:
from sklearn.model_selection import train_test_split
corpus = movie_overviews[["tagline"]].dropna()
corpus = corpus["tagline"]
X_train, X_test = train_test_split(corpus, test_size=0.33, random_state=42)

In [51]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer object
vectorizer = CountVectorizer(lowercase=True, stop_words="english")

# Fit and transform X_train
X_train_bow = vectorizer.fit_transform(X_train)

# Transform X_test
X_test_bow = vectorizer.transform(X_test)

# Print shape of X_train_bow and X_test_bow
print(X_train_bow.shape)
print(X_test_bow.shape)

(4712, 5073)
(2321, 5073)


## Predicting the sentiment of a movie review

### In the previous exercise, you generated the bag-of-words representations for the training and test movie review data. In this exercise, we will use this model to train a Naive Bayes classifier that can detect the sentiment of a movie review and compute its accuracy. Note that since this is a binary classification problem, the model is only capable of classifying a review as either positive (1) or negative (0). It is incapable of detecting neutral reviews.

### In case you don't recall, the training and test BoW vectors are available as X_train_bow and X_test_bow respectively. The corresponding labels are available as y_train and y_test respectively. Also, for you reference, the original movie review dataset is available as df.

### Instructions
-    Instantiate an object of MultinomialNB. Name it clf.
-    Fit clf using X_train_bow and y_train.
-    Measure the accuracy of clf using X_test_bow and y_test.

In [61]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

movie_reviews = pd.read_csv("data/movie_reviews_clean.csv")

X = movie_reviews["review"]
y = movie_reviews["sentiment"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

vectorizer = CountVectorizer(lowercase=True, stop_words="english")
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

In [62]:
# Create a MultinomialNB object
clf = MultinomialNB()

# Fit the classifier
clf.fit(X_train_bow, y_train)

# Measure the accuracy
accuracy = clf.score(X_test_bow, y_test)
print("The accuracy of the classifier on the test set is %.3f" % accuracy)

# Predict the sentiment of a negative review
review = "The movie was terrible. The music was underwhelming and the acting mediocre."
prediction = clf.predict(vectorizer.transform([review]))[0]
print("The sentiment predicted by the classifier is %i" % (prediction))

The accuracy of the classifier on the test set is 0.779
The sentiment predicted by the classifier is 0


## n-gram models for movie tag lines

### In this exercise, we have been provided with a corpus of more than 9000 movie tag lines. Our job is to generate n-gram models up to n equal to 1, n equal to 2 and n equal to 3 for this data and discover the number of features for each model.

### We will then compare the number of features generated for each model.

### Instructions
-    Generate an n-gram model with n-grams up to n=1. Name it ng1
-    Generate an n-gram model with n-grams up to n=2. Name it ng2
-    Generate an n-Gram Model with n-grams up to n=3. Name it ng3
-    Print the number of features for each model.

In [63]:
# Generate n-grams upto n=1
vectorizer_ng1 = CountVectorizer(ngram_range=(1,1))
ng1 = vectorizer_ng1.fit_transform(corpus)

# Generate n-grams upto n=2
vectorizer_ng2 = CountVectorizer(ngram_range=(1,2))
ng2 = vectorizer_ng2.fit_transform(corpus)

# Generate n-grams upto n=3
vectorizer_ng3 = CountVectorizer(ngram_range=(1, 3))
ng3 = vectorizer_ng3.fit_transform(corpus)

# Print the number of features for each model
print("ng1, ng2 and ng3 have %i, %i and %i features respectively" % (ng1.shape[1], ng2.shape[1], ng3.shape[1]))

ng1, ng2 and ng3 have 6614, 37100 and 76881 features respectively


## Higher order n-grams for sentiment analysis

### Similar to a previous exercise, we are going to build a classifier that can detect if the review of a particular movie is positive or negative. However, this time, we will use n-grams up to n=2 for the task.

### The n-gram training reviews are available as X_train_ng. The corresponding test reviews are available as X_test_ng. Finally, use y_train and y_test to access the training and test sentiment classes respectively.

### Instructions
-    Define an instance of MultinomialNB. Name it clf_ng
-    Fit the classifier on X_train_ng and y_train.
-    Measure accuracy on X_test_ng and y_test the using score() method.

In [75]:
movie_reviews = pd.read_csv("data/movie_reviews_clean.csv")

X = movie_reviews["review"]
y = movie_reviews["sentiment"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

ng_vectorizer = CountVectorizer(ngram_range=(1,2))

X_train_ng = ng_vectorizer.fit_transform(X_train)
X_test_ng = ng_vectorizer.transform(X_test)

review = 'The movie was not good. The plot had several holes and the acting lacked panache.'

In [76]:
# Define an instance of MultinomialNB 
clf_ng = MultinomialNB()

# Fit the classifier 
clf_ng.fit(X_train_ng, y_train)

# Measure the accuracy 
accuracy = clf_ng.score(X_test_ng, y_test)
print("The accuracy of the classifier on the test set is %.3f" % accuracy)

# Predict the sentiment of a negative review
review = "The movie was not good. The plot had several holes and the acting lacked panache."
prediction = clf_ng.predict(ng_vectorizer.transform([review]))[0]
print("The sentiment predicted by the classifier is %i" % (prediction))

The accuracy of the classifier on the test set is 0.782
The sentiment predicted by the classifier is 0


## Comparing performance of n-gram models

### You now know how to conduct sentiment analysis by converting text into various n-gram representations and feeding them to a classifier. In this exercise, we will conduct sentiment analysis for the same movie reviews from before using two n-gram models: unigrams and n-grams upto n equal to 3.

### We will then compare the performance using three criteria: accuracy of the model on the test set, time taken to execute the program and the number of features created when generating the n-gram representation.

### Instructions 1/2
-    Initialize a CountVectorizer object such that it generates unigrams.

In [80]:
import time

df = pd.read_csv("data/movie_reviews_clean.csv")

In [81]:
start_time = time.time()
# Splitting the data into training and test sets
train_X, test_X, train_y, test_y = train_test_split(df['review'], df['sentiment'], test_size=0.5, random_state=42, stratify=df['sentiment'])

# Generating ngrams
vectorizer = CountVectorizer(ngram_range=(1,2))
train_X = vectorizer.fit_transform(train_X)
test_X = vectorizer.transform(test_X)

# Fit classifier
clf = MultinomialNB()
clf.fit(train_X, train_y)

# Print accuracy, time and number of dimensions
print("The program took %.3f seconds to complete. The accuracy on the test set is %.2f. The ngram representation had %i features." % (time.time() - start_time, clf.score(test_X, test_y), train_X.shape[1]))

The program took 0.533 seconds to complete. The accuracy on the test set is 0.76. The ngram representation had 79059 features.


### Instructions 2/2
-    Initialize a CountVectorizer object such that it generates ngrams upto n=3.

In [82]:
start_time = time.time()
# Splitting the data into training and test sets
train_X, test_X, train_y, test_y = train_test_split(df['review'], df['sentiment'], test_size=0.5, random_state=42, stratify=df['sentiment'])

# Generating ngrams
vectorizer = CountVectorizer(ngram_range=(1,3))
train_X = vectorizer.fit_transform(train_X)
test_X = vectorizer.transform(test_X)

# Fit classifier
clf = MultinomialNB()
clf.fit(train_X, train_y)

# Print accuracy, time and number of dimensions
print("The program took %.3f seconds to complete. The accuracy on the test set is %.2f. The ngram representation had %i features." % (time.time() - start_time, clf.score(test_X, test_y), train_X.shape[1]))

The program took 1.059 seconds to complete. The accuracy on the test set is 0.77. The ngram representation had 178240 features.
